In [1]:
import cv2
import copy
import time
import tqdm
import pathlib
import random
import warnings
import datetime
import scipy as sp
import pandas as pd
import numpy as np
import scipy.signal
import xgboost as xgb
from time import time
import lightgbm as lgb
from catboost import Pool
from functools import partial
from catboost import CatBoost
from keras.models import Model
from scipy.stats import cauchy
import matplotlib.pyplot as plt
from scipy.stats import laplace
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy.optimize import curve_fit
from keras.layers import Dense, Input
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from scipy.stats import kurtosis, skew
from collections import Counter, defaultdict
from scipy.interpolate import UnivariateSpline
from sklearn.preprocessing import MinMaxScaler
from hyperopt import hp, tpe, Trials, fmin, space_eval
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, average_precision_score, r2_score

np.set_printoptions(precision=5)
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",1000)
pd.set_option('display.max_columns', None)

Using TensorFlow backend.


In [2]:
def pr_auc_metric(y_predicted, y_true):
    return 'pr_auc', average_precision_score(y_true.get_label(), y_predicted), True

def _1gaussian(x, amp1,cen1,sigma1):
    return amp1*(1/(sigma1*(np.sqrt(2*np.pi))))*(np.exp((-1.0/2.0)*(((x-cen1)/sigma1)**2)))

def _1Lorentzian(x, amp, cen, wid):
    return amp*wid**2/((x-cen)**2+wid**2)

In [3]:
filepath = "../input/atma2020529/"
filepath2 = "../input/atma2020529-2/"
filepath3 = "../input/atma20205293/"
filepath4 = "../input/atam202005294/"
train = pd.read_csv(filepath + "train.csv")
test = pd.read_csv(filepath + "test.csv")
fitting = pd.read_csv(filepath4 + "fitting__fixed.csv")
sample_submission = pd.read_csv(filepath + "atmaCup5__sample_submission.csv")
spec_df = pd.read_csv(filepath3 + "spec.csv")
#wave_df = pd.read_csv(filepath2 + "wave_df.csv")
#wave_test = pd.read_csv(filepath2 + "wave_test.csv")
#wave_df = pd.DataFrame(wave_df)
#wave_test = pd.DataFrame(wave_test)
#wave_df = wave_df.iloc[:,:511]

# FE

In [4]:
from tsfresh import extract_features, extract_relevant_features
from tsfresh.feature_extraction import settings
# https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#module-tsfresh.feature_extraction.feature_calculators
fc_parameters = {
    #xx"large_standard_deviation": [{ "r": 0.1}],   
    #xx"ar_coefficient": [{"coeff": 3, "k": 10}], 
    #xx"fft_coefficient": [{"coeff": 3, "attr": "real"}],
    #"cid_ce" :[{"normalize": True}],
    #"autocorrelation": [{"lag": 30}],
    "fft_aggregated" : [{"aggtype": "kurtosis"}],
}
ts_df = extract_features(spec_df, column_id="spectrum_filename", column_sort="wavelength", n_jobs=8, 
                         default_fc_parameters=fc_parameters) #settings.EfficientFCParameters())

ts_df = ts_df.reset_index()
ts_df = ts_df.rename(columns={"id":"spectrum_filename"})

Feature Extraction: 100%|██████████| 40/40 [00:04<00:00,  9.25it/s]


In [5]:
def peak_near_stats(x):
    i = np.argmax(x)
    y = x[i - 10:i + 10]
    return np.sum(y) / x[i], kurtosis(y)

def peak_near_stats2(x):
    i = np.argsort(-x)[1]
    z = x[i - 3:i + 3]
    return np.sum(z) / x[i]

def peak_near_stats4(x):
    i = np.argsort(-x)[3]
    z = x[i - 3:i + 3]
    return np.sum(z) / x[i]

def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

# aggragate spec information
spec_agg = spec_df.groupby("spectrum_filename")["intensity"].agg(["max", "min", "mean", "std"])
spec_agg.columns = ["intensity_" + c for c in spec_agg.columns]
fft = []
peak_near_ratio = []
peak_near1 = []
sec_peak_near1 = []
sec_peak_near2 = []
sec_peak_near3 = []

first_dif_loc = []
first = []
second = []
third = []
diff_kurt = []
diff_max = []

for i, file_df in tqdm(spec_df.groupby("spectrum_filename")):
    x = file_df["wavelength"].values
    y = file_df["intensity"].values
        
    # fast fourier transformation
    F = np.fft.fft(y)
    Amp = np.abs(F)
    fft.append(np.quantile(Amp, 0.95))
    
    # add peak info
    val1, val2 = peak_near_stats(y)
    peak_near_ratio.append(val1)
    peak_near1.append(val2)
    
    val3 = peak_near_stats2(y)
    sec_peak_near1.append(val3)
    val5 = peak_near_stats4(y)
    sec_peak_near3.append(val5)
    
    # savgol_filter
    smooth = scipy.signal.savgol_filter(y, 5, 2, deriv=0),  # なめらかにしただけ   
    first_deriv = scipy.signal.savgol_filter(y, 5, 2, deriv=1),  # 1次微分
    sec_deriv = scipy.signal.savgol_filter(y, 5, 2, deriv=2),
    third_deriv = scipy.signal.savgol_filter(y, 5, 2, deriv=3),
    
    first_dif_loc.append(np.argmax(first_deriv) - np.argmin(first_deriv))
    #first.append(np.max(first_deriv))
    second.append(np.max(sec_deriv) - np.min(sec_deriv))
    #third.append(np.mean(first_deriv))
    
    # diff
    diff_kurt.append((file_df["intensity"].diff(2).diff() + file_df["intensity"].diff(2).diff(-1)).kurt())
    #diff_max.append(abs(file_df["intensity"].diff(2).diff()).max())

spec_agg["amp_0.95"] = fft
spec_agg["peak_ratio"] = peak_near_ratio
spec_agg["peak_near_kur"] = peak_near1
spec_agg["sec_peak_ratio"] = sec_peak_near1
spec_agg["fou_peak_ratio"] = sec_peak_near3

#spec_agg["first_deriv_diff"] = first
spec_agg["sec_deriv_diff"] = second
spec_agg["first_deriv_loc"] = first_dif_loc
#spec_agg["third_deriv_diff"] = third

spec_agg["diff_kurt"] = diff_kurt
#spec_agg["diff_max"] = diff_max

In [6]:
def closest_node(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.abs(nodes - node)
    return np.argmin(dist_2)

def new_peak_near_stats(x, idx):
    y = x[idx - 3:idx + 3]
    return np.sum(y) / x[idx], kurtosis(y)

def transform(df):
    new_df = df.copy()
    # merge original csv and fitting data
    new_df = pd.merge(new_df, fitting, on="spectrum_id", how="left")
    
    # merge original csv and spec information
    new_df= pd.merge(new_df, spec_agg.reset_index(), on="spectrum_filename", how="left")
    
    #new_df = pd.merge(new_df, ts_df, on="spectrum_filename", how="left")
    
    # remove unnecessary columns
    if "target" in new_df.columns:
        new_df = new_df.drop(["spectrum_id", "spectrum_filename", "layout_x","layout_y", "pos_x", 'layout_a'], axis=1)
    else:
        new_df = new_df.drop(["spectrum_id","chip_id","spectrum_filename", "layout_x", "layout_y", "pos_x",'layout_a'], axis=1)
    
    # create new variables
    new_df["ratio2_5"] = new_df["params2"] / new_df["params5"] # I don't know the meaning, but seems effective
    new_df["ratio3_1"] = new_df["params3"] / new_df["params1"] # I don't know the meaning, but seems effective
    new_df["ratio4_6"] = new_df["params4"] / new_df["params6"] # I don't know the meaning, but seems effective
    #new_df["mul0_1"] = new_df["params0"] * new_df["params1"] # I don't know the meaning, but seems effective
    #new_df["ratio0_4"] = new_df["params0"] / new_df["params4"] # I don't know the meaning, but seems effective
    
    return new_df
new_train = transform(train)
new_test = transform(test)
print(f'train shape: {new_train.shape}')
print(f'test shape: {new_test.shape}')

train shape: (7436, 27)
test shape: (6952, 25)


In [7]:
import itertools
def blute_force_fe(df, variables):
    for v in itertools.combinations(variables, 2):
        df["sum"+str(v[0])+"_"+str(v[1])] = df[v[0]] + df[v[1]]
        df["diff"+str(v[0])+"_"+str(v[1])] = df[v[0]] - df[v[1]]
        df["abs_diff"+str(v[0])+"_"+str(v[1])] = np.abs(df[v[0]] - df[v[1]])
        df["mul"+str(v[0])+"_"+str(v[1])] = df[v[0]] * df[v[1]]
        df["div"+str(v[0])+"_"+str(v[1])] = df[v[0]] / df[v[1]]   
    return df

params = ["params0", "params1","params2", "params3","params4", "params5","params6"]
#new_train = blute_force_fe(new_train, params)
#new_test = blute_force_fe(new_test, params)
print(f'train shape: {new_train.shape}')
print(f'test shape: {new_test.shape}')

train shape: (7436, 27)
test shape: (6952, 25)


In [8]:
new_train.head()

,chip_id,exc_wl,target,params0,params1,params2,params3,params4,params5,params6,rms,beta,intensity_max,intensity_min,intensity_mean,intensity_std,amp_0.95,peak_ratio,peak_near_kur,sec_peak_ratio,fou_peak_ratio,sec_deriv_diff,first_deriv_loc,diff_kurt,ratio2_5,ratio3_1,ratio4_6
0,79ad4647da6de6425abf,850,0,30.808589,5.811802e+02,1037.714752,1.531423,22469.651641,1032.317268,8.295610,10.028668,2.521298e-02,1751.0,-228.0,40.292752,172.206792,7643.154333,0.000000,NaN,0.000000,6.591964,302.428571,189,0.034112,1.005229,2.635022e-03,2708.619691
1,79ad4647da6de6425abf,780,0,91.300897,1.740582e+04,1080.510452,4.766233,33257.123175,1077.468855,8.018225,7.948485,3.435612e-01,4219.0,-263.0,166.958984,463.428363,28918.587810,9.569329,-1.257037,5.422615,5.596097,578.285714,-6,0.103895,1.002823,2.738298e-04,4147.691287
2,c695a1e61e002b34e556,780,0,106.642946,1.000000e-10,1119.464438,2.000000,42579.867913,1378.883338,11.687417,10.739859,2.348528e-15,2412.0,-235.0,151.577691,327.857694,18130.782140,12.473051,-1.282283,5.644407,5.754322,337.285714,-7,-0.259870,0.811863,2.000000e+10,3643.223069
3,c695a1e61e002b34e556,780,0,306.933674,1.099486e+04,1139.855067,5.198692,39349.741703,1145.212849,9.445029,10.379948,2.183921e-01,3209.0,-52.0,523.080947,436.481410,22149.537147,12.990825,-1.479866,5.701270,5.879316,373.857143,-10,-0.255881,0.995322,4.728291e-04,4166.185449
4,c695a1e61e002b34e556,780,0,46.133256,2.227622e+04,1120.918337,5.668012,31054.928673,1117.107782,7.658710,8.316550,4.176962e-01,3998.0,-245.0,138.187717,472.009931,30874.913315,10.321911,-1.386428,5.176440,5.750939,491.714286,-7,-0.125171,1.003411,2.544423e-04,4054.851290


# nn modelling

In [9]:
## CNN
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.metrics import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
import tensorflow as tf

#nn_y_train = new_train.target.copy()
#nn_X_train = np.array(wave_df).reshape(-1, 511, 1)
#nn_X_test = np.array(wave_test).reshape(-1, 511, 1)
#nn_y_train = np.array(nn_y_train).reshape(-1, 1)

def sk_pr_auc(y_true, y_pred):
    return tf.py_function(average_precision_score, (y_true, y_pred), tf.float64)

def create_model(input_len, n_filter, filter_size, drop_rate):

    ## 入力の型を定義
    _input = Input((input_len, 1))

    ## 畳み込みブロック(過学習防止にdropoutも追加します)
    x = Conv1D(n_filter, filter_size, padding="same")(_input)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_rate)(x)

    ## 2層目
    x = Conv1D(n_filter, filter_size, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_rate)(x)
    
    ## 3層目
    x = Conv1D(n_filter, filter_size, padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.1)(x)

    ## あとで挙動を確認したいので、この層に名前をつけます
    x = GlobalMaxPool1D(name="hidden")(x)
    x = Dropout(drop_rate)(x)

    ## 0 or 1を当てるのでsigmoidを使います
    _out = Dense(1, activation="sigmoid")(x)

    model = Model(_input, _out, name="Sequential")

    return model

def modelling_nn(tr, target, te):
    X_train = tr.copy()
    y_train = target.copy()
    X_test = te.copy()

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    models = []

    oof = np.zeros(X_train.shape[0])

    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        X_train2 = X_train[train_index,:]
        y_train2 = y_train[train_index]

        X_test2 = X_train[test_index,:]
        y_test2 = y_train[test_index]
        
        clf = create_model(input_len=511, n_filter=64, filter_size=8, drop_rate=0.2)
        cb = EarlyStopping(monitor='auc', patience=3)
        clf.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=[AUC(curve='PR', num_thresholds=1000)])

        history = clf.fit(X_train2,y_train2,
                        validation_data=(X_test2, y_test2),
                        epochs=10,
                        batch_size=16,
                        callbacks = [cb],
                        verbose=1)
            
        oof[test_index] = clf.predict(X_test2)[:,0]
        models.append(clf)

    score = average_precision_score(target, oof)
    print("average precision score = {}".format(score))
    print(confusion_matrix(target, np.round(oof)))
    pred_value = np.zeros(X_test.shape[0])
    for i, model in enumerate(models):
        pred_value += model.predict(X_test)[:,0] / len(models)
    return score, pred_value, oof

#metric_nn, pred_value_nn, oof_nn = modelling_nn(nn_X_train, nn_y_train, nn_X_test)

# hyperopt

In [10]:
def my_hyperopt(X, Y):
    def para_tuning_obj(params):
        params = {
        'boosting_type': 'gbdt', 
        'metric': 'None', 
        'objective': 'binary', 
        "tree_learner": "serial",
        'max_depth': int(params['max_depth']),
        'bagging_freq': int(params['bagging_freq']),
        'bagging_fraction': float(params['bagging_fraction']),
        'num_leaves': int(params['num_leaves']),
        'feature_fraction': float(params['feature_fraction']),
        'learning_rate': float(params['learning_rate']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
        'min_sum_hessian_in_leaf': int(params['min_sum_hessian_in_leaf']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
}
        X.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X.columns]
        real = np.array([])
        pred = np.array([])
        skf = StratifiedKFold(n_splits=5)
        for trn_idx, val_idx in skf.split(X, Y):
            x_train, x_val = X.iloc[trn_idx, :], X.iloc[val_idx, :]
            y_train, y_val = Y.iloc[trn_idx], Y.iloc[val_idx]
            train_set = lgb.Dataset(x_train, y_train, categorical_feature = ['layout_a'])
            val_set = lgb.Dataset(x_val, y_val, categorical_feature = ['layout_a'])
        
            clf = lgb.train(params, train_set, num_boost_round = 100000, early_stopping_rounds = 50, 
                         valid_sets = [train_set, val_set], feval=pr_auc_metric, verbose_eval = 1000)
            pred = np.concatenate((pred, np.array(clf.predict(x_val, num_iteration = clf.best_iteration))), axis=0) 
            real = np.concatenate((real, np.array(y_val)), axis=0) 
        score = average_precision_score(real, pred)
    
        return - score

    trials = Trials()

    space ={
        'max_depth': hp.quniform('max_depth', 1, 5, 1),
        'bagging_freq': hp.quniform('bagging_freq', 1, 10, 1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.2, 1.0),
        'num_leaves': hp.quniform('num_leaves', 5, 30, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.2, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0.01, 1.0),
        'min_data_in_leaf': hp.quniform('min_data_in_leaf', 8, 128, 1),
        'min_sum_hessian_in_leaf': hp.quniform('min_sum_hessian_in_leaf', 5, 30, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0)
    }

    best = fmin(para_tuning_obj, space = space, algo=tpe.suggest, max_evals=50, trials=trials, verbose=1)

    best_params = space_eval(space, best)
    return best_params

#Y = new_train.target.copy()
#X = new_train.drop(["target"], axis=1).copy()
#random_state = 42
#lbl = preprocessing.LabelEncoder()
#lbl.fit(list(X["chip_id"]))
#X["chip_id"] = lbl.transform(list(X["chip_id"]))
#my_hyperopt(X, Y)

# LightGBM

In [11]:
categoricals = []
lgbm_params = {'objective': 'binary', 'metric': 'None', 'boosting_type': 'gbdt', 'tree_learner': 'serial', 'learning_rate': 0.03, "num_leaves": 10, 'random_seed':44,'max_depth': 5}

def modelling_skf(new_train, new_test):
    X_train = new_train.drop(['target'],axis=1).copy()
    y_train = new_train.target.copy()
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(X_train["chip_id"]))
    X_train["chip_id"] = lbl.transform(list(X_train["chip_id"]))
    
    remove_features = []
    for i in X_train.columns:
        if (X_train[i].std() == 0) and i not in remove_features:
            remove_features.append(i)
    X_train = X_train.drop(remove_features, axis=1)
    X_test = new_test.copy()
    X_test = X_test.drop(remove_features, axis=1)
    
    X_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_train.columns]
    X_test.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_test.columns]

    n_folds=5
    skf=StratifiedKFold(n_splits = n_folds, shuffle=True, random_state=0)
    models = []

    valid = np.array([])
    valid_lgb = pd.DataFrame(np.zeros([X_train.shape[0]]))
    real = np.array([])
    features_list = [i for i in X_train.columns if i != "chip_id"]
    feature_importance_df = pd.DataFrame(features_list, columns=["Feature"])
    mean_score = 0
    for i , (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
        print("Fold "+str(i+1))
        X_train2 = X_train.iloc[train_index,:]
        y_train2 = y_train.iloc[train_index]

        X_test2 = X_train.iloc[test_index,:]
        y_test2 = y_train.iloc[test_index]
        
        X_train2.drop(["chip_id"], axis=1, inplace=True)
        X_test2.drop(["chip_id"], axis=1, inplace=True)
        lgb_train = lgb.Dataset(X_train2, y_train2)
        lgb_eval = lgb.Dataset(X_test2, y_test2, reference=lgb_train)
        
        clf = lgb.train(lgbm_params, lgb_train,valid_sets=[lgb_train, lgb_eval],
           num_boost_round=10000,early_stopping_rounds=100,verbose_eval = 1000, feval=pr_auc_metric, categorical_feature = categoricals) 
            
        valid_predict = clf.predict(X_test2, num_iteration = clf.best_iteration)
        mean_score += average_precision_score(y_test2,valid_predict) / n_folds
        valid = np.concatenate([valid, valid_predict])
        valid_lgb.iloc[test_index]  = clf.predict(X_test2, num_iteration = clf.best_iteration).reshape(X_test2.shape[0], 1)
        real = np.concatenate([real, y_test2])
        feature_importance_df["Fold_"+str(i+1)] = clf.feature_importance()
        models.append(clf)
        
    feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
    feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]

    score = average_precision_score(real, valid)
    print("mean score = {}".format(mean_score))
    print("average precision score = {}".format(average_precision_score(real, valid)))
    pred_value = np.zeros(X_test.shape[0])
    for model in models:
        pred_value += model.predict(X_test, num_iteration = model.best_iteration) / len(models)
    return score, pred_value, feature_importance_df, valid_lgb

metric_skf, pred_value_skf, feature_importance_df_skf, _ = modelling_skf(new_train, new_test)

Fold 1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[565]	training's pr_auc: 1	valid_1's pr_auc: 0.94643
Fold 2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[304]	training's pr_auc: 1	valid_1's pr_auc: 0.913304
Fold 3
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[326]	training's pr_auc: 1	valid_1's pr_auc: 0.906596
Fold 4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[206]	training's pr_auc: 0.999917	valid_1's pr_auc: 0.826436
Fold 5
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[411]	training's pr_auc: 1	valid_1's pr_auc: 0.938379
mean score = 0.9062289487775768
average precision score = 0.907421115341871


In [12]:
feature_importance_df_skf.sort_values("Average", ascending=False).reset_index(drop=True)

,Feature,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5,Average,Std,Cv
0,sec_deriv_diff,445,270,306,229,331,316.2,73.007945,0.230892
1,rms,391,144,212,119,232,219.6,95.315476,0.434041
2,diff_kurt,254,149,217,151,242,202.6,44.580713,0.220043
3,fou_peak_ratio,303,152,133,86,197,174.2,73.591847,0.422456
4,peak_near_kur,280,119,117,110,186,162.4,64.926420,0.399793
5,params0,236,151,128,87,207,161.8,53.670849,0.331711
6,ratio3_1,168,172,186,97,164,157.4,31.097267,0.197568
7,params2,204,120,155,85,175,147.8,41.633640,0.281689
8,first_deriv_loc,173,111,135,107,182,141.6,30.968371,0.218703
9,params5,225,106,164,79,125,139.8,50.799213,0.363371


# submission

In [13]:
final_pred = pred_value_skf
score = metric_skf
sample_submission["target"] = final_pred
sample_submission.to_csv("atmacup3_sample_submission"+str(score)[:-10]+".csv", index = False)
sample_submission.head()

,target
0,0.000077
1,0.000176
2,0.000085
3,0.000076
4,0.007146
